# Set up

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.decomposition import NMF, DictionaryLearning, PCA
import deepdish as dd

# Model

In [3]:
#Set up model
numComp=2
initPro='nndsvda'
iterations=10000000
tolerance=1e-6
variable="Amplitude"
dimensons=[50,50]
random=0
NMFmodel = NMF(n_components=numComp, init=initPro, random_state=random, max_iter=iterations,tol=tolerance)

# Load Data

In [4]:
#Use this function to convert the old matlab structs to typical python dictionary 
def matextract(matData,subNames=['Pulse1','Pulse2','Pulse3','Pulse4','Pulse5','Pulse6','Pulse7','Pulse8','Pulse9','Pulse10']):

    #Define output dict
    outData={}

    #For each sample
    for sample in matData:

        print('Loading: '+sample)

        #Further define output
        outData[sample]={}

        #Extarct data
        samp=matData[sample]
        ampData=samp.rawData.amp*10**6
        phiData=samp.rawData.phi
        wData=samp.rawData.w
        qData=samp.rawData.Q

        #Extract meta data
        bounds=samp.Processed.bounds.Data
        pulses=samp.Processed.pulses
        #timePer=samp.Processed.timePerDiv
        #coer=samp.Processed.coercive

        numPulses=np.shape(bounds)[0]

        #For each sub 
        for i in range(0,numPulses):

            #Further define output
            outData[sample][subNames[i]]={}

            outData[sample][subNames[i]]['Amplitude']=[]
            outData[sample][subNames[i]]['Phase']=[]
            outData[sample][subNames[i]]['Resonance']=[]
            outData[sample][subNames[i]]['Q']=[]

            #Get the bounds for the pulse
            a=bounds[i,0]-1
            b=bounds[i,1]

            if sample != 'pmn_28pt_100':
                #Make the time axis
                time=np.linspace(0,b-a-1,b-a)
            else:
                #Have to account for the four missing points in 28pt
                timeA=np.linspace(0,15,16)

                steps=max(0,b-a-20+4)

                timeB=np.linspace(20,b-a-1+4,steps)
                time=np.hstack((timeA,timeB))

            #Pull out datah for the pulse in question
            pulseAmpData=ampData[:,a:b]
            pulsePhiData=phiData[:,a:b]
            pulseWData=wData[:,a:b]
            pulseQData=qData[:,a:b]

            outData[sample][subNames[i]]['Amplitude'].append(pulseAmpData)
            outData[sample][subNames[i]]['Amplitude']=np.squeeze(outData[sample][subNames[i]]['Amplitude'])

            outData[sample][subNames[i]]['Phase'].append(pulsePhiData)
            outData[sample][subNames[i]]['Phase']=np.squeeze(outData[sample][subNames[i]]['Phase'])

            outData[sample][subNames[i]]['Resonance'].append(pulseWData)
            outData[sample][subNames[i]]['Resonance']=np.squeeze(outData[sample][subNames[i]]['Resonance'])

            outData[sample][subNames[i]]['Q'].append(pulseQData)
            outData[sample][subNames[i]]['Q']=np.squeeze(outData[sample][subNames[i]]['Q'])

            outData[sample][subNames[i]]['Time']=time


        #Store INFO data
        outData[sample]['INFO']={}

        outData[sample]['INFO']['Voltages_Applied']=pulses
        #outData[sample]['INFO']['TimePerDivision']=timePer
        #outData[sample]['INFO']['Coercive_Voltage']=coer

    return outData


In [6]:
#Import all the data

dataDir="C:/Users/lgriffin39/Documents/Python Scripts/Data"

pmn_02_2017 = sio.loadmat("pmn_02_2017.mat", squeeze_me=True, struct_as_record=False)["pmn_02_2017"]
pmn_03_2017 = sio.loadmat("pmn_03_2017.mat", squeeze_me=True, struct_as_record=False)["pmn_03_2017"]
pmn_10pt = sio.loadmat("pmn_10pt.mat", squeeze_me=True, struct_as_record=False)["pmn_10pt"]
pmn_28pt_100 = sio.loadmat("pmn_28pt_100.mat", squeeze_me=True, struct_as_record=False)["pmn_28pt_100"]
pmn_28pt_111 = sio.loadmat("pmn_28pt_111.mat", squeeze_me=True, struct_as_record=False)["pmn_28pt_111"]
pmn_36pt_001 = sio.loadmat("pmn_36pt_001.mat", squeeze_me=True, struct_as_record=False)["pmn_36pt_001"]
pmn_36pt_111 = sio.loadmat("pmn_36pt_111.mat", squeeze_me=True, struct_as_record=False)["pmn_36pt_111"]
pmn_Otherside = sio.loadmat("pmn_Otherside.mat", squeeze_me=True, struct_as_record=False)["pmn_Otherside"]
pmn_pt_tetra1 = sio.loadmat("pmn_pt_tetra1.mat", squeeze_me=True, struct_as_record=False)["pmn_pt_tetra1"]


#Define a dictionary of the data to fit
allRaw = {"pmn_02_2017": pmn_02_2017, 
                #"pmn_03_2017" : pmn_03_2017, 
                #"pmn_10pt" : pmn_10pt, 
                #"pmn_28pt_100" : pmn_28pt_100, 
                #"pmn_28pt_111" : pmn_28pt_111, 
                "pmn_36pt_001" : pmn_36pt_001, 
                #"pmn_36pt_111" : pmn_36pt_111, 
                #"pmn_Otherside" : pmn_Otherside, 
                "pmn_pt_tetra1" : pmn_pt_tetra1}

allRaw=matextract(allRaw)

Loading: pmn_02_2017
Loading: pmn_36pt_001
Loading: pmn_pt_tetra1


# Accessing data

In [ ]:
plt.plot(allRaw['pmn_02_2017']['Pulse4']['Amplitude'][230,:])
plt.show()